In [1]:
import pickle
import mediapipe as mp
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
model=pickle.load(open('counter_model.pkl','rb'))
scaler=pickle.load(open('counter_scaler.pkl','rb'))

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
import cv2
import time

In [16]:
cap=cv2.VideoCapture(0)

counter=0
stage=None
probab=0

with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  
    time.sleep(5)
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        if results.pose_landmarks is not None:
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks.landmark[25].visibility >= 0.3:   #checking visibilty of left knee

                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles
                    .get_default_pose_landmarks_style())
            
                keypoints=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
                scaled_keypoints=scaler.transform(keypoints.reshape(1,-1))
                curr_stage=model.predict(scaled_keypoints)[0]

                if(curr_stage=='down'):
                    probab=model.predict_proba(scaled_keypoints)[0][0]
                else:
                    probab=model.predict_proba(scaled_keypoints)[0][1]                    

                if(curr_stage=='up' and stage== 'down' and probab> 0.88):
                    stage=curr_stage
                    counter+=1
                    print(counter)
                else:
                    if(probab>0.89):
                        stage=curr_stage
        
        cv2.rectangle(image, (0,0), (380,60), (245,117,16), -1)

        #rep data
        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(counter), (6,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #stage data
        cv2.putText(image, 'STAGE', (130,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, stage, (90,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #probab data
        cv2.putText(image, 'Probability', (260,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(probab), (255,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        cv2.imshow('pose',image)

        k = cv2.waitKey(1) & 0xFF
        if(k == 27):
            break
cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
